In [1]:
import asyncio
import datetime
import ssl
import os
import pickle

In [2]:
class test():
    def __init__(self, a):
        self.a = a
        self.b = 5
        
    def mult(self):
        print(self.a*self.b)

In [3]:
class result():
    def __init__(self, c):
        self.c = c
        
    def disp(self):
        print(f'Result is {self.c}')
        self.c = c

In [4]:
async def tcp_echo_client():
    ctx = ssl.SSLContext(protocol=ssl.PROTOCOL_TLS_CLIENT)
    ctx.load_cert_chain('oemCertChain.pem', 'oemLeaf.key', '12345')
    ctx.load_verify_locations(cafile='v2gRootCACert.pem')
    ctx.check_hostname = False
    ctx.verify_mode = ssl.VerifyMode.CERT_REQUIRED
    ctx.set_ciphers('ECDHE-ECDSA-AES256-GCM-SHA384:ECDHE-RSA-AES256-GCM-SHA384')
    reader, writer = await asyncio.open_connection(
        '127.0.0.1', 9002, ssl=ctx)
    
    i = 0
    while (i < 5):
        dt = datetime.datetime.now()
        data = test(i)
        message = pickle.dumps(data)
        print(f'Time: {dt} Send: {message!r}')
        print(f'a: {data.a}, b: {data.b}')
        writer.write(message)
        await writer.drain()

        
        message = await reader.read(100)
        dt = datetime.datetime.now()
        data = pickle.loads(message)
        print(f'Time: {dt} Received: {message!r}')
        data.disp()
        i = i + 1
        await asyncio.sleep(1)
    
    message = 'quit'
    print(f'Send: {message!r}')
    writer.write(message.encode())
    await writer.drain()
    data = await reader.read(100)
    print(f'Received: {data.decode()!r}')
        
    print('Close the connection')
    writer.close()
    await writer.wait_closed()

In [5]:
await tcp_echo_client()

Time: 2023-03-08 21:59:47.210789 Send: b'\x80\x04\x95)\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x04test\x94\x93\x94)\x81\x94}\x94(\x8c\x01a\x94K\x00\x8c\x01b\x94K\x05ub.'
a: 0, b: 5
Time: 2023-03-08 21:59:47.214762 Received: b'\x80\x04\x95#\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x06result\x94\x93\x94)\x81\x94}\x94\x8c\x01c\x94Nsb.'
Result is None


NameError: name 'c' is not defined